In [ ]:
with open("input.txt") as f:
    raw = f.read().splitlines()


In [ ]:
import re

regex = re.compile("(?:(\$) )?(\w+) *(.+)?")


In [ ]:
from dataclasses import dataclass, field
from enum import Enum


class FileType(Enum):
    REGULAR = "file"
    DIRECTORY = "dir"


@dataclass
class Node:
    name: str
    type: FileType
    size: int = 0
    child: list = field(default_factory=list)

    def append(self, child: FileType) -> None:
        if self.type is not FileType.DIRECTORY:
            raise TypeError("Parent is not a directory")
        self.child.append(child)

    def __json__(self):
        return self.__dict__


In [ ]:
fs = None
stack: list[Node] = []

for line in raw:
    a, b, c = regex.match(line).groups()
    if a == "$":
        if b == "cd":
            if c == "/":
                fs = Node("/", FileType.DIRECTORY)
                stack.append(fs)
                continue
            if c == "..":
                stack.pop()
                continue
            else:
                n = Node(c, FileType.DIRECTORY)
                stack[-1].append(n)
                stack.append(n)
        if b == "ls":
            continue
    else:
        if b == "dir":
            continue
        n = Node(c, FileType.REGULAR, int(b))
        stack[-1].append(n)

In [ ]:
print(fs)

In [ ]:
from functools import reduce


def update_size(fs: Node) -> None:
    folders = list(filter(lambda f: f.type == FileType.DIRECTORY, fs.child))
    for f in folders:
        update_size(f)
    fs.size = reduce(lambda sum, f: sum + f.size, fs.child, 0)


In [ ]:
update_size(fs)
print(fs)


In [ ]:
threshold = 100000


def get_treshold_sum(node: Node, threshold: int) -> int:
    totalsum = 0
    for child in node.child:
        if child.type is not FileType.DIRECTORY:
            continue
        if child.size <= threshold:
            totalsum += child.size
        totalsum += + get_treshold_sum(child, threshold)
    return totalsum


print(get_treshold_sum(fs, threshold))


In [ ]:
total = 70000000
update_size = 30000000

freespace = total - fs.size
needed = update_size - freespace

In [ ]:

def find_small(root: Node) -> Node:
    small = root
    for child in root.child:
        if child.type is not FileType.DIRECTORY:
            continue
        if child.size < needed:
            continue
        if not small or child.size >= needed:
            sm = find_small(child)
            if sm and sm.size < small.size:
                small = sm
    return None if small.size < needed else small
print(find_small(fs).size)